In [5]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [28]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re


def delevel_query(query_list):
    line_level = []
    sub_query = []
    pos_delete, pos_where = len(query_list), len(query_list)
    
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete = i
        if line.startswith('WHERE'):
            pos_where = i 
        if line.startswith('FROM'):
            pos_from = i-1
        # if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
        #     pos_join = i
  
    for line in query_list[:pos_from+2]:
        line_level.append((line, "level_1"))
        
    for line in query_list[pos_from+1:pos_where]:
        if line.startswith(' ') or line.startswith('FROM'):
            sub_query.append(line[3:])
        else:
            line_level.append((line, "level_1"))

    for line in query_list[pos_where:pos_delete]:    
        line_level.append((line, "level_1"))

    return line_level, sub_query[1:]


def parse_sub_query(sub_query_list):
    sub_query = "\n".join(sub_query_list)
    sub_query = sub_query.lstrip('\n').lstrip(' ')
    
    formatter = column_parser.Parser(sub_query)
    formatted = formatter.format_query(sub_query)
    sub_query_list = formatted.split('\n')

    return sub_query_list


def has_child(sub_query_list):
    query_list = parse_sub_query(sub_query_list)
    query_list = delevel_query(query_list)[1]
    return query_list, query_list != []


# def main():
#     query = """SELECT * FROM sfdc.accounts sfdc_accounts
#             LEFT JOIN (SELECT MAX(dt) FROM (SELECT dt FROM sfdc.oppty sfdc_oppty) LEFT JOIN (SELECT dt FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
#             WHERE dt > '2020-04-03' 
#             """

#     formatter = column_parser.Parser(query)
#     formatted = formatter.format_query(query)
#     query_list = formatted.split('\n')

#     has_subquery = True
#     while has_subquery:
#         sub_query_list, has_subquery = has_child(query_list)
#         query_list = sub_query_list
#         print(has_subquery, '\n'.join(query_list),  '\n')

In [6]:
query = """SELECT * FROM sfdc.accounts sfdc_accounts
LEFT JOIN (SELECT MAX(dt) FROM (SELECT dt FROM sfdc.oppty sfdc_oppty) LEFT JOIN (SELECT dt FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03' 
"""


In [7]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
cte_queries = formatter.parse_cte(formatted_query)

In [8]:
print(formatted_query)

SELECT *
FROM sfdc.accounts sfdc_accounts
LEFT JOIN
  (SELECT MAX(dt)
   FROM
     (SELECT dt
      FROM sfdc.oppty sfdc_oppty)
   LEFT JOIN
     (SELECT dt
      FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03'


In [52]:
query_list = formatted_query.split('\n')
has_subquery = True
counter = 0
while has_subquery:
    counter += 1
    sub_query_list, has_subquery = has_child(query_list)
    query_list = sub_query_list
    print(has_subquery, counter, '\n'.join(query_list),  '\n')

True 1 SELECT MAX(dt)
FROM
  (SELECT dt
   FROM sfdc.oppty sfdc_oppty)
LEFT JOIN
  (SELECT dt
   FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt 

True 2 SELECT dt
FROM sfdc.oppty sfdc_oppty)
SELECT dt
FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt 

False 3  



In [53]:
query_list_0 = formatted_query.split('\n')

In [54]:
query_list_1, has_subquery = has_child(query_list_0)
query_list_1

['SELECT MAX(dt)',
 'FROM',
 '  (SELECT dt',
 '   FROM sfdc.oppty sfdc_oppty)',
 'LEFT JOIN',
 '  (SELECT dt',
 '   FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt']

In [55]:
query_list_2, has_subquery = has_child(query_list_1)
query_list_2

['SELECT dt',
 'FROM sfdc.oppty sfdc_oppty)',
 'SELECT dt',
 'FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt']

In [58]:
query_list_2 = parse_sub_query(query_list_1)


In [59]:
query_list_2

['SELECT MAX(dt)',
 'FROM',
 '  (SELECT dt',
 '   FROM sfdc.oppty sfdc_oppty)',
 'LEFT JOIN',
 '  (SELECT dt',
 '   FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt']

In [60]:
query_list_2 = delevel_query(query_list_2)[1]

In [61]:
query_list_2

['SELECT dt',
 'FROM sfdc.oppty sfdc_oppty)',
 'SELECT dt',
 'FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt']

## delevel_query

In [ ]:
def delevel_query(query_list):
    line_level = []
    sub_query = []
    pos_delete, pos_where = len(query_list), len(query_list)
    
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete = i
        if line.startswith('WHERE'):
            pos_where = i 
        if line.startswith('FROM'):
            pos_from = i-1
        # if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
        #     pos_join = i
  
    for line in query_list[:pos_from+2]:
        line_level.append((line, "level_1"))
        
    for line in query_list[pos_from+1:pos_where]:
        if line.startswith(' ') or line.startswith('FROM'):
            sub_query.append(line[3:])
        else:
            line_level.append((line, "level_1"))

    for line in query_list[pos_where:pos_delete]:    
        line_level.append((line, "level_1"))

    return line_level, sub_query[1:]

In [79]:
query_list = parse_sub_query(query_list_1)

In [96]:
for i, line in enumerate(query_list):
    print(i, line)

0 SELECT MAX(dt)
1 FROM
2   (SELECT dt
3    FROM sfdc.oppty sfdc_oppty)
4 LEFT JOIN
5   (SELECT dt
6    FROM sfdc.cases)) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt


In [101]:
line_level = []
sub_query = []
pos_delete, pos_where = len(query_list)-1, len(query_list)-1
pos_join = []
for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete = i
    if line.startswith('WHERE'):
        pos_where = i 
    if line.startswith('FROM'):
        pos_from = i-1
        pos_join.append(i+1)
    if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
        pos_join.append(i+1)
        
pos_join.append(pos_delete)     


In [102]:
pos_delete, pos_where, pos_from, pos_join

(6, 6, 0, [2, 5, 6])

In [ ]:
# for line in query_list[:pos_from+2]:
#     line_level.append((line, "level_1"))

# for line in query_list[pos_from+1:pos_where]:
#     if line.startswith(' ') or line.startswith('FROM'):
#         sub_query.append(line[3:])
#     else:
#         line_level.append((line, "level_1"))

pos_join_list = iter(pos_join)



for line in query_list[pos_where:pos_delete]:    
    line_level.append((line, "level_1"))
    